In [4]:
import os
from pathlib import Path

DATASET_ID = '369'

SRC_FOLDER = os.path.join(
    Path().resolve().parent,
    'nnunet'
)

os.environ['nnUNet_raw'] = f"{SRC_FOLDER}/nnUNet_raw"
os.environ['nnUNet_preprocessed'] = f"{SRC_FOLDER}/nnUNet_preprocessed"
os.environ['nnUNet_results'] = f"{SRC_FOLDER}/nnUNet_results"

In [ ]:
!nnUNetv2_plan_and_preprocess -d 369 --verify_dataset_integrity -np 4 -pl nnUNetPlannerResEncM

Fingerprint extraction...
Dataset369
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100%|███████████████████████████████████████████| 40/40 [00:07<00:00,  5.64it/s]
Experiment planning...
Attempting to find 3d_lowres config. 
Current spacing: [2.5        0.82681653 0.82681653]. 
Current patch size: (40, 224, 192). 
Current median shape: [ 97.         497.08737864 497.08737864]
Attempting to find 3d_lowres config. 
Current spacing: [2.5        0.85162102 0.85162102]. 
Current patch size: (40, 224, 192). 
Current median shape: [ 97.         482.60910548 482.60910548]
Attempting to find 3d_lowres config. 
Current spacing: [2.5        0.87716966 0.87716966]. 
Current patch size: (40, 224, 192). 
Current median shape: [ 

In [3]:
!nnUNetv2_train 369 '2d' 4 -device 'cpu' --npz -p nnUNetResEncUNetMPlans

Traceback (most recent call last):
  File "/Users/keithchiang/opt/miniconda3/envs/nnunetv2/bin/nnUNetv2_train", line 8, in <module>
    sys.exit(run_training_entry())
  File "/Users/keithchiang/opt/miniconda3/envs/nnunetv2/lib/python3.10/site-packages/nnunetv2/run/run_training.py", line 275, in run_training_entry
    run_training(args.dataset_name_or_id, args.configuration, args.fold, args.tr, args.p, args.pretrained_weights,
  File "/Users/keithchiang/opt/miniconda3/envs/nnunetv2/lib/python3.10/site-packages/nnunetv2/run/run_training.py", line 196, in run_training
    nnunet_trainer = get_trainer_from_args(dataset_name_or_id, configuration, fold, trainer_class_name,
  File "/Users/keithchiang/opt/miniconda3/envs/nnunetv2/lib/python3.10/site-packages/nnunetv2/run/run_training.py", line 64, in get_trainer_from_args
    plans = load_json(plans_file)
  File "/Users/keithchiang/opt/miniconda3/envs/nnunetv2/lib/python3.10/site-packages/batchgenerators/utilities/file_and_folder_operations.py